In [38]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV


In [27]:
def preProcessing(img):
    # Check if the image is already grayscale
    if len(img.shape) == 2:
        gray = img
    else:
        # Convert the image to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blurring with a kernel size of 5x5 and sigma of 0
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # Resize the image
    resized = cv2.resize(gray, (500, 300))

    # Apply histogram equalization to enhance the contrast of the image
    hist = cv2.equalizeHist(resized)

    return hist


In [28]:
def edgeDetection(img):
    return cv2.Canny(img, 100, 200)

In [29]:
def resizeImage(img):
    return cv2.resize(img , (128*4 , 64*4))

In [30]:
orb = cv2.ORB_create()

In [31]:
# Prepare the training data
X = []
y = []
D = []
L = []
for i in range(0, 6):
    for j in range(1, 6):
        filename = 'images/{}_men ({}).JPG'.format(i, j)
        img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
        if not img is None:
            gray = preProcessing(img)
        else:
            continue    
        if gray is None:
            print("gray is None")

        edgedImage = edgeDetection(gray)
        finalImage = resizeImage(edgedImage)
        keypoints, descriptors = orb.detectAndCompute(finalImage, None)
        D.append(descriptors)
        L.append(i*np.ones(len(descriptors)))
        filename = 'images/{}_woman ({}).JPG'.format(i, j)
        img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
        if not img is None:
            gray = preProcessing(img)
        else:
            continue    
        if gray is None:
            print("gray is None")

        edgedImage = edgeDetection(gray)
        finalImage = resizeImage(edgedImage)
        keypoints, descriptors = orb.detectAndCompute(finalImage, None)
        D.append(descriptors)
        L.append(i*np.ones(len(descriptors)))
        


In [32]:
X = np.vstack(D)
Y = np.hstack(L)

In [33]:
# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [39]:
param_dist = {'n_estimators': randint(100, 1000),
              'max_depth': randint(5, 50),
              'min_samples_split': randint(2, 10),
              'min_samples_leaf': randint(1, 5),
              'max_features': ['auto', 'sqrt', 'log2']}

In [40]:
rfc = RandomForestClassifier(random_state=42)

In [41]:
rfc_random = RandomizedSearchCV(estimator=rfc, param_distributions=param_dist,
                               n_iter=100, cv=5, random_state=42)

In [42]:
rfc_random.fit(X_train, Y_train)

KeyboardInterrupt: 

In [ ]:
best_params = rfc_random.best_params_
print(best_params)

In [ ]:
# Use best hyperparameters to train classifier
clf = RandomForestClassifier(n_estimators=best_params['n_estimators'],
                             max_depth=best_params['max_depth'],
                             min_samples_split=best_params['min_samples_split'],
                             min_samples_leaf=best_params['min_samples_leaf'],
                             max_features=best_params['max_features'],
                             random_state=42)

In [ ]:
# Train the classifier on the training data
clf.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [ ]:
# Make predictions on the testing data
y_pred = clf.predict(X_test)

In [ ]:
# Evaluate the performance of the classifier
score = clf.score(X_test, Y_test)
print("Accuracy:", score)

Accuracy: 0.29613970588235294
